## Initial setup

In [191]:
import os

if not os.path.exists("balatro-ABC.wiki"):
    !git clone https://github.com/Aurif/balatro-ABC.wiki.git

## Pre-generation clean up

In [192]:
import shutil

!cd balatro-ABC.wiki && git pull

if os.path.exists("balatro-ABC.wiki/Docs"):
    shutil.rmtree("balatro-ABC.wiki/Docs")
os.makedirs("balatro-ABC.wiki/Docs", exist_ok=True)

Updating 9e90496..17b2940
Fast-forward
 Home.md                         |   2 +-
 How-to-create-a-joker.md        | 191 ++++++++++++++++++++++++++++++++++++++++
 How-to-create-your-first-mod.md |   2 +-
 3 files changed, 193 insertions(+), 2 deletions(-)


## Generation

In [193]:
from collections import defaultdict
from typing import Optional
import uuid
import re

class MarkdownConverter:
    def __init__(self, transforms: dict[str, callable]):
        self.transforms = transforms

    def convert(self, content: str) -> str:
        content = re.sub(r"\*\*([^*]+)\*\*", lambda m: self.transforms['bold'](m.group(1)), content)
        content = re.sub(r"\*([^*]+)\*", lambda m: self.transforms['italic'](m.group(1)), content)
        content = re.sub(r"`([^`]+)`", lambda m: self.transforms['code'](m.group(1)), content)
        content = re.sub(r"\[([^\]]+)\]\(([^)]+)\)", lambda m: self.transforms['url'](m.group(1), m.group(2)), content)
        content = re.sub(r"(\\?\n)", self.transforms['newline'], content)
        content = re.sub(r"\"", self.transforms['"'], content)
        return content
    
    
MARKDOWN_TO_HTML = MarkdownConverter({
    'bold': lambda x: f"<strong>{x}</strong>",
    'italic': lambda x: f"<em>{x}</em>",
    'code': lambda x: f"<code>{x}</code>",
    'url': lambda name, url: f'<a href="{url}">{name}</a>',
    'newline': '\n',
    '"': '&quot;'
})
    
MARKDOWN_TO_TOOLTIP = MarkdownConverter({
    'bold': lambda x: x,
    'italic': lambda x: x,
    'code': lambda x: x,
    'url': lambda name, url: name,
    'newline': '&#10;',
    '"': '&quot;'
})

In [194]:
class DocPage:
    PAGES = {}
    def __init__(self, name: str):
        self.name = name
        self.sections = defaultdict(list)

    @classmethod
    def get_page(cls, name: str) -> 'DocPage':
        if name not in cls.PAGES:
            cls.PAGES[name] = cls(name)
        return cls.PAGES[name]

    @classmethod
    def generate_all(cls) -> None:
        for page in cls.PAGES.values():
            page.generate_page()

    def generate_page(self) -> None:
        with open(f"balatro-ABC.wiki/Docs/{self.name}.md", "w") as file:
            file.write(self._generate_table_of_contents())
            file.write(self._generate_page_contents())

    def _generate_page_contents(self) -> str:
        output = ""
        for section_name, methods in self.sections.items():
            output += f"# {section_name}\n\n" + "".join([m.to_markdown() for m in methods])
        return output

    def _generate_table_of_contents(self) -> str:
        output = '<!-- -->\n<!-- Generated automatically based on lua files, DO NOT EDIT MANUALLY -->\n<!-- -->\n\n# Table of contents\n<table role="table">'
        for section_name, methods in self.sections.items():
            output += f'\n<thead><tr><th colspan="2"><h3>{section_name}</h3></th></tr></thead>\n<tbody>\n'
            output += "\n".join([m.to_table_row() for m in methods])
            output += "\n</tbody>\n"
        output += "</table>\n\n"
        return output

class DocMethod:
    def __init__(self, name: str, page_name: str, section_name: str):
        self.name = name
        self.page_name = page_name
        self.section_name = section_name
        self.description = ""
        self.links = ""
        self.params = {}
        self.returns = None
        self.is_constructor = False
        self.is_field = False

    def set_params(self, params: list[str]) -> None:
        for p in params:
            self.params[p.strip()] = DocParam(p.strip(), "unknown", "")

    def register(self) -> None:
        page = DocPage.get_page(self.page_name)
        page.sections[self.section_name].append(self)

    def to_markdown(self) -> str:
        output = f"### `{self.get_full_name()}`\n"
        output += self.description.strip() if len(self.description)>0 else "*\<description not given>*"
        output += "".join([f"\n- {p.to_full()}" for p in self.params.values()])
        if self.returns is not None:
            output += f"\n- returns {self.returns.to_full()}"
        if len(self.links) > 0:
            output += f"\n\n{self.links.strip()}"
        output += "\n<br><br>\n\n"
        return output
    
    def to_table_row(self) -> str:
        one_line_summary = MARKDOWN_TO_HTML.convert(self.description.strip()).split("\n")[0]
        tooltip_summary = MARKDOWN_TO_TOOLTIP.convert(self.description.strip())
        return f'<tr><td><a href="#{self.get_hash_url()}" title="{tooltip_summary}"><code>{self.get_full_name()}</code></a></td> <td>{one_line_summary}</td></tr>'
    
    def get_full_name(self) -> str:
        if self.is_field:
            return f".{self.name}"
        params = ", ".join([p.name for p in self.params.values()])
        name = (":" if not self.is_constructor else "") + f"{self.name}({params})"
        if self.returns is not None:
            name += " : " + self.returns.name
        return name
    
    def get_hash_url(self) -> str:
        return re.sub(r"[^\w-]", "", re.sub(r" ", "-", self.get_full_name()))

class DocParam:
    def __init__(self, name: str, param_type: str, description: str):
        self.name = name
        self.param_type = param_type
        self.description = description

    def to_short(self) -> str:
        return f"{self.name}: {self.param_type}"
    
    def to_full(self) -> str:
        return f"`{self.to_short()}`" + (f" - {self.description}" if len(self.description)>0 else "")

In [195]:
def parse_directory(directory: str) -> None:
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                parse_file(content)

def parse_file(contents: str) -> None:
    sections = re.split(r'\n\s*\n\s*---\s*\n---\s*(.+)\n---\s*\n', contents)
    if len(sections) == 1:
        return
    
    for s in range(1, len(sections), 2):
        parse_section(sections[s+1], sections[s])

def parse_section(contents: str, name: str) -> None:
    for constructor_doc in re.findall(r'---@class\s+(.+?)\s*(?::.+)?\n((?:---.*\n)+)---@overload\s+fun\(([^)]+)\)(?::\s*\S+)\s*(.*)', contents):
        comments = [c.strip() for c in constructor_doc[1][3:].split("\n---") if c not in ("---", "***", "")]
        parse_constructor(comments, constructor_doc[0], constructor_doc[2], name, constructor_doc[3])

    for method_doc in re.findall(r'((?:---.+\n)+)function ([^:]+):([^(]+)\((.*)\)', contents):
        comments = [c.strip() for c in method_doc[0][3:].split("\n---") if c not in ("---", "***", "")]
        parse_method(comments, *method_doc[1:], name)

def parse_method(comments: str, prefix: str, name: str, params: str, section_name: str) -> None:
    if any([c in ["@private"] for c in comments]) or (not prefix.startswith("ABC.") and not prefix.startswith("__ABC.")):
        return
    
    doc = DocMethod(name, ".".join(prefix.split(".")[1:]), section_name)
    doc.set_params(params.split(",") if len(params) > 0 else [])
    type_overwrites = {}
    for c in comments:
        if re.search(r'^\[[^\]]+\]\([^)]+\)$', c):
            doc.links += "\n" + c
        elif c.startswith("@generic"):
            c = re.match(r'@generic\s+([^:]+?)\s*(?::\s*(.+))?', c)
            type_overwrites[c.group(1)] = c.group(2) or "unknown"
        elif c.startswith("@param"):
            c = sanitize_types(c)
            if c[1] == "self":
                continue
            doc.params[c[1]] = DocParam(c[1], type_overwrites.get(c[2], c[2]), " ".join(c[3:]))
        elif c.startswith("@return"):
            c = sanitize_types(c)
            if c[2] == "self":
                continue
            doc.returns = DocParam(c[2], type_overwrites.get(c[1], c[1]), " ".join(c[3:]))
        elif c[0] != "@":
            doc.description += "\n" + c
    doc.register()

def parse_constructor(comments: str, name: str, params: str, section_name: str, constructor_comments: str) -> None:
    for c in comments:
        if c.startswith("@field"):
            parse_field(c, name, section_name)

    if any([c in ["@private"] for c in comments]) or not name.startswith("ABC.") or "private" in constructor_comments:
        return

    param_types = {}
    for p in params.split(","):
        p_name, p_type = p.split(":")
        param_types[p_name.strip()] = p_type.strip()

    doc = DocMethod(name, name[4:], section_name)
    doc.is_constructor = True
    for c in comments:
        if re.search(r'^\[[^\]]+\]\([^)]+\)$', c):
            doc.links += "\n" + c
        elif c.startswith("\\@*param*"):
            c = re.match(r'\\@\*param\*\s*`(.+)`\s*—\s*(.+)', c)
            if not c:
                continue
            p_name = c.group(1)
            doc.params[p_name] = DocParam(p_name, param_types.get(p_name, "unknown"), c.group(2))
        elif c[0] != "@" and not c.startswith("\@"):
            doc.description += "\n" + c
    doc.register()

def parse_field(comment: str, name: str, section_name: str) -> None:
    comment = sanitize_types(comment)
    if comment[1] == "private" or len(comment) <= 3:
        return

    doc = DocMethod(comment[1], name[4:], section_name)
    doc.description = " ".join(comment[3:]).replace("<br/>", "\\\n")
    doc.is_field = True
    doc.register()

def sanitize_types(content: str) -> list[str]:
    content = re.sub(r'(\S+)\|{[^}]+}', lambda m: m.group(1), content)

    repl_map = {}
    def repl_func(m: re.match) -> str:
        uid = str(uuid.uuid4())
        repl_map[uid] = m.group(0)
        return uid
    content = re.sub(r'fun\([^)]*\):\s*\S+', repl_func, content)

    return [repl_map.get(c, c) for c in content.split(" ")]

parse_directory("..\ABC")
DocPage.generate_all()

### Sidebar

In [196]:
class SidebarSection:
    SIDEBAR_ROOT = None
    def __init__(self, name: str):
        self.name = name
        self.content = {}

    def under_root(self) -> None:
        self.get_section("").content[self.name] = self

    @classmethod
    def get_section(cls, path: str) -> 'SidebarSection':
        if SidebarSection.SIDEBAR_ROOT is None:
            SidebarSection.SIDEBAR_ROOT = SidebarSection("")
        if path == "":
            return SidebarSection.SIDEBAR_ROOT
        
        path = path.split(".")
        parent = SidebarSection.get_section(".".join(path[:-1]))
        if path[-1] not in parent.content:
            parent.content[path[-1]] = SidebarSection(path[-1])
        return parent.content[path[-1]]
    
    def render_contents(self, indent: int) -> str:
        spacing = " "*(2*indent)
        name_spacing = "&nbsp;"*(3*indent)
        result = f"{spacing}<details open>\n{spacing}  <summary>{name_spacing}{self.name}</summary>\n{spacing}  <p> </p>\n"
        result += self.render_contents_inner(indent)
        result += f"{spacing}</details>\n"
        return result
    
    def render_contents_inner(self, indent: int) -> str:
        return "".join([c.render_contents(indent+1) for c in self.content.values()])
    
    @classmethod
    def render_sidebar(cls) -> None:
        with open(f"balatro-ABC.wiki/_Sidebar.md", "w") as file:
            file.write("<!-- -->\n<!-- Generated automatically based on lua files, DO NOT EDIT MANUALLY -->\n<!-- -->\n\n")
            file.write("<h3><div>\n"+cls.SIDEBAR_ROOT.render_contents_inner(-1)+"</div></h3>")

class SidebarLink:
    def __init__(self, path: str, display_name: Optional[str] = None, *, parent_prefix: str = ""):
        self.path = path
        self.display_name = display_name or self.path.split(".")[-1]
        path = path.split(".")
        if parent_prefix != "":
            path = [parent_prefix, *path]
        parent = SidebarSection.get_section(".".join(path[:-1]))
        parent.content[path[-1]] = self

    def render_contents(self, indent: int) -> str:
        return f'{"  "*indent}<p>{"&nbsp;"*3*(indent+1)}<a href="https://github.com/Aurif/balatro-ABC/wiki/{self.path}">{self.display_name}</a></p>\n'


In [197]:
def find_how_tos() -> list[tuple[str, str]]:
    results = []
    for root, dirs, files in os.walk("balatro-ABC.wiki"):
        for file in files:
            if not re.match(r"How-to-.+\.md", file):
                continue

            order = float('inf')
            file_path = os.path.join(root, file)
            with open(file_path, 'r', encoding='utf-8') as content:
                header = content.readline()
                match = re.match(r"<!-- Order: (\d+) -->", header)
                if match:
                    order = float(match.group(1))
                
            results.append((file[:-3], file[7:-3].replace("-", " ").capitalize(), order))

    results.sort(key=lambda x: x[2])
    return [(r[0], r[1]) for r in results]

In [198]:
SidebarLink("", "Getting started")

SidebarSection("How to").under_root()
for path, name in find_how_tos():
    SidebarLink(path, name, parent_prefix="How to")

SidebarSection("Documentation").under_root()
for key in DocPage.PAGES.keys():
    SidebarLink(key, parent_prefix="Documentation")

SidebarSection.render_sidebar()


## Commiting docs

In [199]:
response = input("Do you want to commit generated docs? (yes/no): ").strip().lower()
if response != "yes":
    print("Commit cancelled.")
    raise SystemExit

!cd balatro-ABC.wiki && git add -A . && git commit -m "[Autogenerated docs]" && git push

[master d5358ac] [Autogenerated docs]
 3 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 .img/how_to__joker__file_structure.png
 create mode 100644 .img/how_to__joker__ingame.png
 create mode 100644 .img/how_to__joker__undiscovered.png


To https://github.com/Aurif/balatro-ABC.wiki.git
   17b2940..d5358ac  master -> master
